<a href="https://colab.research.google.com/github/fcoelhomrc/MachineLearning/blob/main/TAAPC_Exercises/week4_NLP/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2Vec

In this notebook, you'll implement the **skip-gram** version of the Word2Vec model that was presented in the lecture. Please refer to the slides and to the original paper (https://arxiv.org/abs/1301.3781) whenever needed.

We'll use the WikiText2 dataset, which is a collection of over 100 million tokens extracted from the set of verified Good and Featured articles on Wikipedia. 

To split the text into words, we shall use the 'basic_english' tokenizer from torchtext (https://pytorch.org/text/stable/index.html). This is a word-level tokenizer that splits a text into a list of English uncased words and punctuation symbols. Note that a special '\<unk>' token is used whenever an unknown word is found.

In [11]:
!pip install torchdata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import numpy as np
import torch
from torchtext.datasets import WikiText2
from torchtext.data import get_tokenizer
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from tqdm import tqdm

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd

train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')

You should start by building `vocab`, a dictionary containing the vocabulary. The keys will be the words and the values are the corresponding indices, starting from 0. It is a good practice to discard rare words from the vocabulary, so we'll only keep those words that appear at least `MIN_REPS` times.

Thus, you should do the following in the cell below:

1. Build a `word_count` dictionary containing the number of times that each word appears. For this purpose, you should create an iterator from `train_iter` and iterate on it to get each sequence in the dataset. Each sequence should be split in a list of tokens (words) by calling the `tokenizer`. 

2. Build a `vocab` dictionary as described above. The `vocab` should only contain valid words that appear at least `MIN_REPS`. We have reserved a special index (-100) for the '\<unk>' token which you should use for out-of-vocabulary words.
    
It is also useful to have the inverse mapping from `vocab`, i.e. another dictionary that, given the word index, outputs the corresponding word. We have already built that for you in `vocab_inv`.

**Remark:** We could be using `torchtext.vocab`, which facilitates building the `vocab` dictionary and its inverse. For pedagogical reasons, this time you will implement these by yourself.

In [13]:
MIN_REPS = 50 

### YOUR CODE HERE ###
word_count = {}
for sequence in train_iter:
    tokens = tokenizer(sequence)
    for idx, tk in enumerate(tokens):
        if tk not in word_count:
            word_count[tk] = 1
        else:
            word_count[tk] += 1
vocab = {}
idx = 0
for tk, count in word_count.items():
    if count >= MIN_REPS and tk != "<unk>":
        vocab[tk] = idx
        idx += 1

### *** ###
vocab['<unk>'] = -100
vocab_inv = {val: key for (key, val) in vocab.items()}

print(f'Vocabulary has {len(vocab)-1} different words.')

Vocabulary has 4098 different words.


In [14]:
for i in range(10):
    print(vocab_inv.get(i, "KEY_ERROR"))

=
valkyria
chronicles
iii
no
3
(
japanese
,
.


Now it is time to complete the `SkipGramDataset` class by implementing its `__init__` method. This method receives `train_iter`, `vocab`, and the width of the window for the skip gram data. For each word, you will consider all the words that appear up to `window_width` words **before or after** the word, so the **total sequence length will be `2*window_width + 1`**.

In this `__init__` method you should build (input, context) pairs for the skip gram model and store them in the list attributes `self.inputs` and `self.contexts`.

In [15]:
class SkipGramDataset(Dataset):
    def __init__(self, train_iter, vocab, window_width):
        ### YOUR CODE HERE ###
        self.inputs = []
        self.contexts = []
        unk_idx = vocab["<unk>"]
        for sequence in train_iter:
            tokens = tokenizer(sequence)
            length = len(tokens)
            for i, tk in enumerate(tokens):
                if i < window_width or i >= length - window_width:
                    continue
                for j in range(i - window_width, i + window_width + 1):
                    if j == i:
                        continue
                    next = tokens[j]
                    self.inputs.append(vocab.get(tk, unk_idx))
                    self.contexts.append(vocab.get(next, unk_idx))
        ### *** ###
        
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):
        return self.inputs[index], self.contexts[index]
        

In [16]:
WINDOW_WIDTH = 4

dataset = SkipGramDataset(train_iter, vocab, WINDOW_WIDTH)
print(f'Dataset has {len(dataset)} examples.')

Dataset has 14980944 examples.


Let's look at some word pairs...

In [17]:
for i in range(200, 250):
    x, y = dataset[i]
    x_word = vocab_inv[x]
    y_word = vocab_inv[y]
    print(x_word, y_word)


is iii
is outside
is japan
is ,
is a
is <unk>
is role
is @-@
a outside
a japan
a ,
a is
a <unk>
a role
a @-@
a playing
<unk> japan
<unk> ,
<unk> is
<unk> a
<unk> role
<unk> @-@
<unk> playing
<unk> video
role ,
role is
role a
role <unk>
role @-@
role playing
role video
role game
@-@ is
@-@ a
@-@ <unk>
@-@ role
@-@ playing
@-@ video
@-@ game
@-@ developed
playing a
playing <unk>
playing role
playing @-@
playing video
playing game
playing developed
playing by
video <unk>
video role


Let's implement the word2vec model. As you can see from the lecture slides, this model simply consists of an `nn.Embedding` layer mapping word indices to embeddings and a `nn.Linear` projection layer mapping to the vocabulary dimension. As you may already know, you should **not** apply the softmax at the output. The loss function you will use does that for you and in a more numerically stable way.

In [18]:
class SkipGramModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, embed_max_norm=None):
        super().__init__()
        ### YOUR CODE HERE ###
        self.embedding = nn.Embedding(vocab_size, embed_dim,
                                      max_norm=embed_max_norm)
        self.linear = nn.Linear(in_features=embed_dim,
                                out_features=vocab_size)       
        ### *** ###
    
    def forward(self, x):
        '''
        Input:
          x: torch.LongTensor - tensor with shape (batch,) containing the indices of the input words
        Return value:
          logits: torch.FloatTensor - tensor with shape (batch, vocab_size) containing unnormalized probabilities
            for each word in the vocabulary
        '''
        ### YOUR CODE HERE ###
        # oov input means output word probabilities is a zero vector
        oov_idx = (x < 0).nonzero()
        x[oov_idx] = 0 # temporary
        embedded = self.embedding(x)
        logits = self.linear(embedded)
        logits[oov_idx] = 0
        ### *** ###
        return logits


Implement the `fit()` function in the cell below. This function should return two lists, containing the loss and accuracy values over the training epochs.

In [19]:
def fit(model, train_loader, optimizer, **kwargs):
    
    num_epochs = kwargs.get('num_epochs', 100)
    loss_fn = kwargs.get('loss_fn', nn.functional.cross_entropy)
    device = kwargs.get('device', torch.device('cpu'))
    
    model.train()
    train_loss_hist, train_acc_hist = [], []
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}')
        pbar = tqdm(train_loader, total=len(train_loader))
        train_loss, train_acc = 0, 0
        for x, y in pbar:
            ### YOUR CODE HERE ###
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(x)
            loss = loss_fn(logits, y)
            loss.backward()
            optimizer.step()
            with torch.no_grad():
                preds = logits.argmax(dim=1)
            acc = (preds == y).float().sum() / ((y != -100).float().sum() + 1e-6)
            ### *** ###
            train_loss += loss.item()
            train_acc += acc.item()
            pbar.set_description(f'loss = {loss:.3f} | acc = {acc:.3f}')
        train_loss /= len(train_loader)
        train_acc /= len(train_loader)
        print(f'train loss = {train_loss:.3f} | train acc = {train_acc:.3f}')
        train_loss_hist.append(train_loss)
        train_acc_hist.append(train_acc)
        
    return train_loss_hist, train_acc_hist

Now we instantiate the model, the optimizer and the dataloader and we're ready to train! As usual, we need to define a few hyperparameters. We set `EMBEDDING_DIM = 300` by default as this was the value used by the authors in the paper, but later you can play around with different values. Limiting the embeddings norm by setting `EMBEDDING_MAX_NORM = 1` is optional, but you can verify empirically that it will yield nicer embeddings. 

In [25]:
EMBEDDING_DIM = 300
EMBEDDING_MAX_NORM = 1
BATCH_SIZE = 128
NUM_EPOCHS = 5
LEARNING_RATE = 1e-4

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
# elif torch.backends.mps.is_available():
#     DEVICE = torch.device('mps')
else:
    DEVICE = torch.device('cpu')
print('DEVICE:', DEVICE)

model = SkipGramModel(vocab_size=len(vocab)-1, embed_dim=EMBEDDING_DIM, embed_max_norm=EMBEDDING_MAX_NORM).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

DEVICE: cpu


In [26]:
train_loss, train_acc = fit(model, dataloader, optimizer, num_epochs=NUM_EPOCHS, device=DEVICE)

Epoch 0


loss = 6.586 | acc = 0.092:   3%|▎         | 4045/117039 [04:05<1:54:10, 16.50it/s]


KeyboardInterrupt: ignored

In [ ]:
plt.title('Train loss')
plt.plot(range(len(train_loss)), train_loss)
plt.show()
plt.title('Train accuracy')
plt.plot(range(len(train_acc)), train_acc)
plt.show()

In [ ]:
torch.save(model.state_dict(), 'word2vec.pt')

Extract the embeddings from the model and normalize them to have unit L2-norm. Store the normalized embeddings in a `np.array` named `embeddings_norm`.

In [2]:
### YOUR CODE HERE ###

### *** ###

Now we'll **visualize the learned embeddings**! Since these are 300-dimensional, they are impossible to visualize directly. First, **we need to project the embeddings into a two- (or three-)dimensional space**. We do this by using the t-Distributed Stochastic Neighbor Embedding algorithm (**t-SNE**, https://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf), which usually provides very nice visualizations of high-dimensional data.

In [3]:
# get embeddings
embeddings_df = pd.DataFrame(embeddings)

# t-SNE transform
tsne = TSNE(n_components=2)
embeddings_df_trans = tsne.fit_transform(embeddings_df)
embeddings_df_trans = pd.DataFrame(embeddings_df_trans)

# get token order
embeddings_df_trans.index = [vocab_inv[i] for i in range(len(vocab_inv)-1)]

# if token is a number
is_numeric = embeddings_df_trans.index.str.isnumeric()

NameError: ignored

Run the cell below and open the generated file (word2vec_visualization.html) in your browser. Explore the embedding space. You should see semantically related words close to each other.

In [4]:
color = np.where(is_numeric, "green", "black")
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=embeddings_df_trans[0],
        y=embeddings_df_trans[1],
        mode="text",
        text=embeddings_df_trans.index,
        textposition="middle center",
        textfont=dict(color=color),
    )
)
fig.write_html("./word2vec_visualization.html")

NameError: ignored

Implement the function `get_most_similar`, which receives a `word` and outputs a dictionary where the keys are the `topN` words whose embeddings are closer (as measured by the cosine similarity) to the embedding of the `word`. The values of the returned dictionary should be filled with the similarities.

In [ ]:
def get_most_similar(word, vocab, vocab_inv, embeddings, topN=1):
    ### YOUR CODE HERE ###
    
    ## *** ###
    return topN_dict

In [ ]:
for word in ['mother', 'portugal', 'queen', 'sports']:
    print(f"Top-10 words most similar to '{word}': {get_most_similar(word, vocab, vocab_inv, embeddings_norm, topN=10)}\n")

## Negative sampling

Our vocabulary has only ~4k words, so training without negative sampling was doable. However, it is very common to have much larger vocabularies (10^5-10^7 words) where the original word2vec model is much more expensive to train.

For this reason, you'll now implement the skip-gram Word2Vec model with negative sampling. You should start by implementing a new dataset class in the cell below, which extends the original `SkipGramDataset`. The `SkipGramDatasetNS` should have the following attributes:
1. `inputs: List[int]` - same as before
2. `contexts: torch.LongTensor` - tensor with shape `(num_examples, num_neg + 1)` that contains the index of the positive word (true context) and the indices of the negative words, which are sampled from the vocabulary according to the distribution provided in the list `word_probs`. You can use the function `np.random.choice` to do the sampling.
3. `labels: torch.FloatTensor` - tensor with shape (num_neg + 1) that is 1 in the position corresponding to the positive word and 0 elsewhere.

**Remark:** We could be generating negative examples *on-the-fly* (i.e., in `__getitem__`) instead of generating them only once in `__init__`. This way we would avoid repeating the same negative examples every training epoch, which could improve the results a bit. However, `np.random.choice` is quite slow and therefore on-the-fly generation would slow down training considerably.

In [ ]:
class SkipGramDatasetNS(SkipGramDataset):
    def __init__(self, train_iter, vocab, word_probs, window_width, num_neg):
        super().__init__(train_iter, vocab, window_width)
        ### YOUR CODE HERE ###
        
        ### *** ###
    
    def __getitem__(self, index):
        '''
        Return values:
          input: int - the index of the input word 
          contexts: torch.LongTensor - tensor with shape (num_neg + 1,) containing the indices of the positive
            and negative words
          labels: torch.FloatTensor - tensor with binary labels with the same shape as contexts
        '''
        return self.inputs[index], self.contexts[index], self.labels

We can think of many distributions to sample our negatives from. The authors of wav2vec found empirically that a good option is to sample negative words $w$ from the distribution $q$ defined by: $$q(w) = \frac{\hat{p}(w)^{3/4}}{Z},$$ where $\hat{p}$ is the distribution of words in the training corpus and $Z$ is a normalization constant (such that $\sum_{\forall w \in V} q(w) = 1$). The fractional exponent (3/4) has a smoothing effect on the distribution, so that we avoid sampling the most frequent words too often.

In the cell below, build the list `word_probs` where `word_probs[i]`$= q(w_i)$.

In [ ]:
### YOUR CODE HERE ###

### *** ###

Now we instantiate our dataset. We just need to decide the number of negative words to use. A small value (<10) is usually enough.

In [ ]:
NUM_NEG = 5

datasetNS = SkipGramDatasetNS(train_iter, vocab, word_probs, WINDOW_WIDTH, NUM_NEG)
print(f'Dataset has {len(datasetNS)} examples.')

The model architecture and the `fit` routine also need to be changed. How? That's what you need to think about out now :)

In [ ]:
class SkipGramModelNS(nn.Module):
    def __init__(self, vocab_size, embed_dim, embed_max_norm=None):
        super().__init__()
        ### YOUR CODE HERE ###
        
        ### *** ###
    
    def forward(self, x, c):
        '''
        Inputs:
          x: torch.LongTensor - tensor with shape (batch,) containing the indices of the input words
          c: torch.LongTensor - tensor with shape (batch, num_context) containing the indices of the context words
        Return value:
          sim: torch.FloatTensor - tensor with shape (batch, num_context) containing the cosine similarities between
            the embeddings of x and the embeddings of each context word in c.
        '''
        ### YOUR CODE HERE ###
        
        ### *** ###
        return sim


In [ ]:
def fitNS(model, train_loader, optimizer, **kwargs):
    ### YOUR CODE HERE ###
    
    ### *** ###
        
    return train_loss_hist, train_acc_hist

Now, we're ready to go! Due to negative sampling, this model should train a bit faster than the previous one.

In [ ]:
modelNS = SkipGramModelNS(vocab_size=len(vocab)-1, embed_dim=EMBEDDING_DIM, embed_max_norm=EMBEDDING_MAX_NORM).to(DEVICE)
optimizerNS = optim.Adam(modelNS.parameters(), lr=LEARNING_RATE)
dataloaderNS = DataLoader(datasetNS, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
train_lossNS, train_accNS = fitNS(modelNS, dataloaderNS, optimizerNS, num_epochs=NUM_EPOCHS, device=DEVICE)

In [ ]:
plt.title('Train loss (with neg. sampling)')
plt.plot(range(len(train_lossNS)), train_lossNS)
plt.show()
plt.title('Train accuracy (with neg. sampling)')
plt.plot(range(len(train_accNS)), train_accNS)
plt.show()

In [ ]:
torch.save(model.state_dict(), 'word2vecNS.pt')

Now we repeat the experiments we have done before for the model without negative sampling. You can copy the code for normalizing the embeddings and paste it in the cell below.

In [ ]:
### YOUR CODE HERE ###

### *** ###

In [ ]:
# get embeddings
embeddings_df = pd.DataFrame(embeddings)

# t-SNE transform
tsne = TSNE(n_components=2)
embeddings_df_trans = tsne.fit_transform(embeddings_df)
embeddings_df_trans = pd.DataFrame(embeddings_df_trans)

# get token order
embeddings_df_trans.index = [vocab_inv[i] for i in range(len(vocab_inv)-1)]

# if token is a number
is_numeric = embeddings_df_trans.index.str.isnumeric()

In [ ]:
color = np.where(is_numeric, "green", "black")
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=embeddings_df_trans[0],
        y=embeddings_df_trans[1],
        mode="text",
        text=embeddings_df_trans.index,
        textposition="middle center",
        textfont=dict(color=color),
    )
)
fig.write_html("./word2vecNS_visualization.html")

In [ ]:
for word in ['mother', 'portugal', 'queen', 'sports']:
    print(f"Top-10 words most similar to '{word}': {get_most_similar(word, vocab, vocab_inv, embeddings_norm, topN=10)}")

## Playground

Now you can have some fun finding other interesting relations in the embeddings you have learned. There are also a few hyperparameters that we have defined rather arbitrarily that you can play with.
With little effort, you can also implement the **continuous bag-of-words (CBOW)** version of Word2Vec.